# Основы парсинга

Автор: Булыгин Олег
* [Я в LinkedIn](https://www.linkedin.com/in/obulygin/)  
* [Мой канал в ТГ по Python](https://t.me/pythontalk_ru)
* [Чат канала](https://t.me/pythontalk_chat)
* [Блог в Телетайпе](https://teletype.in/@pythontalk)

In [1]:
import pandas as pd
import requests

In [2]:
# метод get
res = requests.get('https://netology.ru/blog/')
# res
res.status_code

200

In [3]:
# браузер отрисовал бы страницу на основе данного текста
res.text

'<!DOCTYPE html>\n<html lang="ru-RU" prefix="og: https://ogp.me/ns#">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="https://gmpg.org/xfn/11">\n<title>Медиа Нетологии: об образовании в диджитале</title>\n\n<meta name="description" content="Издание об онлайн-образовании и карьере в ИТ. Пишем о маркетинге, управлении проектами, дизайне, веб-разработке, Data Science и образовании." />\n<meta name="robots" content="max-image-preview:large" />\n<link rel="canonical" href="https://netology.ru/blog/" />\n<link rel="next" href="https://netology.ru/blog/page/2" />\n<meta property="og:locale" content="ru_RU" />\n<meta property="og:site_name" content="Нетология: образовательная платформа" />\n<meta property="og:type" content="article" />\n<meta property="og:title" content="Медиа Нетологии об образовании и карьере в диджитале 🎓" />\n<meta property="og:description" content="Полезное о маркетинге, управлении проектами,

In [5]:
# получаем плохой статус
bad_request = requests.get('https://netology.ru/blog/some_page')
bad_request

<Response [404]>

In [6]:
bad_request.text

'<!DOCTYPE html>\n<html lang="ru-RU" prefix="og: https://ogp.me/ns#">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="https://gmpg.org/xfn/11">\n<title>Страница не найдена &#8212; Медиа Нетологии</title>\n\n<meta name="robots" content="noindex" />\n<link rel="canonical" href="https://netology.ru/blog/some_page" />\n\n<link rel="preload" href="https://netology.ru/blog/wp-content/plugins/rate-my-post/public/css/fonts/ratemypost.ttf" type="font/ttf" as="font" crossorigin="anonymous">\n\n<script data-cfasync="false" data-pagespeed-no-defer type="text/javascript">//<![CDATA[\n\tvar gtm4wp_datalayer_name = "dataLayer";\n\tvar dataLayer = dataLayer || [];\n//]]>\n</script>\n<link rel="dns-prefetch" href="//netology.ru" />\n<link rel="dns-prefetch" href="//fonts.googleapis.com" />\n<link rel="dns-prefetch" href="//s.w.org" />\n<link rel="dns-prefetch" href="//blog-prod-bucket.website.yandexcloud.net" />\n<link rel

In [7]:
# как разбирать всю эту разметку? Поможет BeautifulSoup.
from bs4 import BeautifulSoup

In [11]:
# создаем объекта, через методы которого будем искать нужные теги и извлекать их содержимое
soup = BeautifulSoup(res.text)
soup

<!DOCTYPE html>
<html lang="ru-RU" prefix="og: https://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<title>Медиа Нетологии: об образовании в диджитале</title>
<meta content="Издание об онлайн-образовании и карьере в ИТ. Пишем о маркетинге, управлении проектами, дизайне, веб-разработке, Data Science и образовании." name="description"/>
<meta content="max-image-preview:large" name="robots"/>
<link href="https://netology.ru/blog/" rel="canonical"/>
<link href="https://netology.ru/blog/page/2" rel="next"/>
<meta content="ru_RU" property="og:locale"/>
<meta content="Нетология: образовательная платформа" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="Медиа Нетологии об образовании и карьере в диджитале 🎓" property="og:title"/>
<meta content="Полезное о маркетинге, управлении проектами, дизайне, веб-разработке, Data Science и образова

In [28]:
# функция find_all позволяет найти все указанные тег с нужными атрибутами (с вложениями), возвращает список
news = soup.find_all('div','posts__item')
print(len(news))
print(news[0])

0


IndexError: list index out of range

In [14]:
for el in news:
    title = el.find('a', 'posts__link').text
    print(title)
    link = el.find('a', 'posts__link').get('href')
    print(link)
    date = el.find('div', 'posts__date').text
    print(date)
    category = el.find('span').text
    print(category)
    print()

Напишем функцию на основе всех предыдущих действий, которая будет возвращать датафрейм с датой, заголовком, категорией и ссылкой на полный текст поста


In [16]:
def get_netology_blog_posts():
    url = 'https://netology.ru/blog/'
    req = requests.get(url).text
    soup = BeautifulSoup(req)

    news = soup.find_all('div', class_='posts__item')

    netology_blog = pd.DataFrame()

    for el in news:
        title = el.find('a', 'posts__link').text
        link = el.find('a', 'posts__link').get('href')
        date = el.find('div', 'posts__date').text
        category = el.find('a', 'tags__item').text
        row = {'date': date, 'title': title, 'link': link, 'category':category}
        netology_blog = pd.concat([netology_blog, pd.DataFrame([row])])
    return netology_blog.reset_index(drop=True)

get_netology_blog_posts()

,date,title,link,category
0,1 декабря 2023,Как научиться говорить «нет» и не испытывать ч...,https://netology.ru/blog/12-2023-how-to-say-no,Разобраться
1,30 ноября 2023,10 фишек Телеграма для экологичного общения по...,https://netology.ru/blog/11-2023-fishki-telegrama,Продуктивность
2,29 ноября 2023,"Спецпроекты брендов: сколько стоят, как реализ...",https://netology.ru/blog/11-2023-special-projects,Разобраться
3,23 ноября 2023,Как преодолеть страх и заговорить на иностранн...,https://netology.ru/blog/11-2023-yazykovoy-barer,Продуктивность
4,21 ноября 2023,"7 книг, которые помогут научиться говорить «не...",https://netology.ru/blog/11-2023-books-about-r...,Книжная полка
5,16 ноября 2023,Что такое зерокодинг,https://netology.ru/blog/11-2023-what-is-zeroc...,Современные профессии
6,15 ноября 2023,Сколько стоит запустить первый онлайн-курс: ра...,https://netology.ru/blog/11-2023-edm-how-much-...,Разобраться
7,10 ноября 2023,Насмотренность в дизайне: зачем изучать дизайн...,https://netology.ru/blog/11-2023-visual-experi...,Разобраться
8,8 ноября 2023,Найти и обезвредить: разбираем стадии професси...,https://netology.ru/blog/11-2023-stadii-vygora...,Продуктивность
9,3 ноября 2023,10 книг о развитии памяти и внимания,https://netology.ru/blog/11-2023-books-about-m...,Книжная полка


А теперь поработаем поиском по сайту

In [17]:
# cформируем поисковый запрос, обратите внимание на его формат
URL = 'https://netology.ru/blog/?s=python'

In [18]:
req = requests.get(URL)
req.text

'<!DOCTYPE html>\n<html lang="ru-RU" prefix="og: https://ogp.me/ns#">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="https://gmpg.org/xfn/11">\n<title>python | Медиа Нетологии</title>\n\n<meta name="robots" content="max-image-preview:large" />\n<link rel="canonical" href="https://netology.ru/blog?s=python" />\n<link rel="next" href="https://netology.ru/blog/page/2?s=python" />\n\n<link rel="preload" href="https://netology.ru/blog/wp-content/plugins/rate-my-post/public/css/fonts/ratemypost.ttf" type="font/ttf" as="font" crossorigin="anonymous">\n\n<script data-cfasync="false" data-pagespeed-no-defer type="text/javascript">//<![CDATA[\n\tvar gtm4wp_datalayer_name = "dataLayer";\n\tvar dataLayer = dataLayer || [];\n//]]>\n</script>\n<link rel="dns-prefetch" href="//netology.ru" />\n<link rel="dns-prefetch" href="//fonts.googleapis.com" />\n<link rel="dns-prefetch" href="//s.w.org" />\n<link rel="dns-prefetch

В функцию get можно передавать параметры и заголовки запроса в виде словаря

In [19]:
# в request можно передать параметры запроса и заголовки (headers) в виде словарей.
URL = 'https://netology.ru/blog/'
params = {
    's': 'python'
}

req = requests.get(URL, params=params)

req.text

'<!DOCTYPE html>\n<html lang="ru-RU" prefix="og: https://ogp.me/ns#">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="https://gmpg.org/xfn/11">\n<title>python | Медиа Нетологии</title>\n\n<meta name="robots" content="max-image-preview:large" />\n<link rel="canonical" href="https://netology.ru/blog?s=python" />\n<link rel="next" href="https://netology.ru/blog/page/2?s=python" />\n\n<link rel="preload" href="https://netology.ru/blog/wp-content/plugins/rate-my-post/public/css/fonts/ratemypost.ttf" type="font/ttf" as="font" crossorigin="anonymous">\n\n<script data-cfasync="false" data-pagespeed-no-defer type="text/javascript">//<![CDATA[\n\tvar gtm4wp_datalayer_name = "dataLayer";\n\tvar dataLayer = dataLayer || [];\n//]]>\n</script>\n<link rel="dns-prefetch" href="//netology.ru" />\n<link rel="dns-prefetch" href="//fonts.googleapis.com" />\n<link rel="dns-prefetch" href="//s.w.org" />\n<link rel="dns-prefetch

In [20]:
soup = BeautifulSoup(req.text)
articles = soup.find_all('article', class_='status-publish')
len(articles)

6

Напишем функция для извлечения постов по несколькоим запросам со страницы

In [22]:
import time  # будем ставить задержку между запросами

def get_netology_posts(query):
    netology_blog = pd.DataFrame()
    URL = 'https://netology.ru/blog/'
    for q in query:
        params = {
            's': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='status-publish')
        for article in articles:
            title = article.find('h2', class_='entry-title').text
            link = article.find('h2', class_='entry-title').find('a').get('href')
            date = article.find('span', class_='posted-on').text.strip()
            category = article.find('div', class_='entry-cats').text
            row = {'date': date, 'title': title, 'link': link, 'category':category}
            netology_blog = pd.concat([netology_blog, pd.DataFrame([row])])
    return netology_blog.reset_index(drop=True)

get_netology_posts(['python', 'data science'])

AttributeError: 'NoneType' object has no attribute 'text'

In [24]:
def get_netology_posts(query):
    netology_blog = pd.DataFrame()
    for q in query:
        URL = 'https://netology.ru/blog/'
        params = {
            's': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='status-publish')
        for article in articles:
            title = article.find('h2', class_='entry-title').text
            link = article.find('h2', class_='entry-title').find('a').get('href')
            date = article.find('span', class_='posted-on').text.strip()
            if article.find('div', class_='entry-cats'):
                category = article.find('div', class_='entry-cats').text
            else:
                category = 'Новость'
            row = {'date': date, 'title': title, 'link': link, 'category':category}
            netology_blog = pd.concat([netology_blog, pd.DataFrame([row])])
    return netology_blog.reset_index(drop=True)

res = get_netology_posts(['python', 'data science'])
res

,date,title,link,category
0,4 апреля 2023,Кто такой fullstack-разработчик на Python,https://netology.ru/blog/04-2023-fullstack-python,Современные профессии
1,12 октября 2021,Python занял первое место в рейтинге языков пр...,https://netology.ru/blog/news/12-10-2021-python,Новость
2,31 июля 2019,С чего начать изучение Python: 3 первых шага,https://netology.ru/blog/07-2019-python-3-perv...,Современные профессии
3,15 марта 2019,С чего начать изучение Python: книги для начин...,https://netology.ru/blog/03-2019-python-knigi-...,Книжная полка
4,16 января 2019,"Как перейти в Machine learning, если знаешь Py...",https://netology.ru/blog/2019-01-machine-learn...,Карьера
5,15 января 2019,Создание модели распознавания лиц с использова...,https://netology.ru/blog/2019-01-modeli-raspoz...,Руководство
6,16 января 2023,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/01-2023-data-science-...,Книжная полка
7,15 декабря 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/12-2022-data-science-...,Книжная полка
8,15 ноября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/11-2022-data-science-...,Книжная полка
9,12 октября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/10-2022-data-science-...,Книжная полка


А если хотим добавить в датафрейм полные тексты всех постов?

In [25]:
def add_full_text(posts_df):
    i = 0
    for el in posts_df['link']:
        # print(el)
        req = requests.get(el).text
        soup = BeautifulSoup(req)
        time.sleep(0.3)
        full_text = soup.find('div', class_='entry-content').text.strip()
        posts_df.loc[i, 'text'] = full_text
        i += 1
    return posts_df

add_full_text(res)

,date,title,link,category,text
0,4 апреля 2023,Кто такой fullstack-разработчик на Python,https://netology.ru/blog/04-2023-fullstack-python,Современные профессии,"На старте бывает сложно определиться с тем, ка..."
1,12 октября 2021,Python занял первое место в рейтинге языков пр...,https://netology.ru/blog/news/12-10-2021-python,Новость,Впервые за 20 лет Python стал самым популярным...
2,31 июля 2019,С чего начать изучение Python: 3 первых шага,https://netology.ru/blog/07-2019-python-3-perv...,Современные профессии,"В прошлой статье мы рассказали, почему стоит и..."
3,15 марта 2019,С чего начать изучение Python: книги для начин...,https://netology.ru/blog/03-2019-python-knigi-...,Книжная полка,"Нетология рассказала, с какой книги начать изу..."
4,16 января 2019,"Как перейти в Machine learning, если знаешь Py...",https://netology.ru/blog/2019-01-machine-learn...,Карьера,Директор разработки и data scientist ДомКлик.р...
5,15 января 2019,Создание модели распознавания лиц с использова...,https://netology.ru/blog/2019-01-modeli-raspoz...,Руководство,"Адаптировали статью Файзана Шайха о том, как с..."
6,16 января 2023,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/01-2023-data-science-...,Книжная полка,"Подушка, которая избавляет от храпа с помощью ..."
7,15 декабря 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/12-2022-data-science-...,Книжная полка,"Определяем пневмонию по кашлю с помощью ИИ, см..."
8,15 ноября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/11-2022-data-science-...,Книжная полка,"В новом дайджесте рассказываем, как ИИ взламыв..."
9,12 октября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/10-2022-data-science-...,Книжная полка,"Делаем покемонов из кого угодно, смотрим «Ла-Л..."


In [ ]:
URL = 'https://netology.ru/blog/?infinity=scrolling'
params = {
    'page': 1,
    'query_args[s]': 'python'
}
req = requests.get(URL, params=params)
req.text

'{"type":"success","html":"\\n<article id=\\"post-42088\\" class=\\"post-42088 post type-post status-publish format-standard has-post-thumbnail hentry category-professions tag-dev\\">\\n\\t<a class=\\"thumb-link\\" href=\\"https:\\/\\/netology.ru\\/blog\\/04-2023-fullstack-python\\"><span role=\\"img\\" aria-label=\\"\\" class=\\"featured-image has-background-cover has-animation\\" style=\\"background-image: url(\'https:\\/\\/blog-prod-bucket.website.yandexcloud.net\\/uploads\\/2023\\/04\\/\\u0424\\u0443\\u043b\\u0441\\u0442\\u044d\\u043a-980x460.jpg\');\\"><\\/span><\\/a>\\t<div class=\\"hentry-inner\\">\\n        <span class=\\"posted-on\\">\\n            <a href=\\"https:\\/\\/netology.ru\\/blog\\/04-2023-fullstack-python\\" rel=\\"bookmark\\">4 \\u0430\\u043f\\u0440\\u0435\\u043b\\u044f 2023<\\/a>\\n        <\\/span>\\n        <header class=\\"entry-header primary-font\\">\\n\\t\\t<div class=\\"entry-cats\\"><a href=\\"https:\\/\\/netology.ru\\/blog\\/category\\/professions\\" rel=

In [ ]:
req.json()['html']

'\n<article id="post-42088" class="post-42088 post type-post status-publish format-standard has-post-thumbnail hentry category-professions tag-dev">\n\t<a class="thumb-link" href="https://netology.ru/blog/04-2023-fullstack-python"><span role="img" aria-label="" class="featured-image has-background-cover has-animation" style="background-image: url(\'https://blog-prod-bucket.website.yandexcloud.net/uploads/2023/04/Фулстэк-980x460.jpg\');"></span></a>\t<div class="hentry-inner">\n        <span class="posted-on">\n            <a href="https://netology.ru/blog/04-2023-fullstack-python" rel="bookmark">4 апреля 2023</a>\n        </span>\n        <header class="entry-header primary-font">\n\t\t<div class="entry-cats"><a href="https://netology.ru/blog/category/professions" rel="category tag">Современные профессии</a></div><h2 class="entry-title"><a href="https://netology.ru/blog/04-2023-fullstack-python" rel="bookmark">Кто такой fullstack-разработчик на Python</a></h2>\t\t</header><!-- .entry-h

In [ ]:
URL = 'https://netology.ru/blog/?infinity=scrolling'
params = {
    'page': 2,
    'query_args[s]': 'python'
}
req = requests.get(URL, params=params)

soup = BeautifulSoup(req.json()['html'])
articles = soup.find_all('article', class_='status-publish')
netology_blog = pd.DataFrame()
for article in articles:
    title = article.find('h2', class_='entry-title').text
    link = article.find('h2', class_='entry-title').find('a').get('href')
    date = article.find('span', class_='posted-on').text.strip()
    if article.find('div', class_='entry-cats'):
        category = article.find('div', class_='entry-cats').text
    else:
        category = 'Новость'
    row = {'date': date, 'title': title, 'link': link, 'category':category}
    netology_blog = pd.concat([netology_blog, pd.DataFrame([row])])
netology_blog

,date,title,link,category
0,14 октября 2016,"Что такое Python: чем он хорош, где пригодится...",https://netology.ru/blog/python,Разобраться
0,30 октября 2023,"Как начать карьеру в IT тем, кто считает себя ...",https://netology.ru/blog/10-2023-start-v-it,Карьера Разобраться
0,26 октября 2023,Как расти в карьере горизонтально,https://netology.ru/blog/10-2023-how-to-grow-h...,Карьера Разобраться
0,22 сентября 2023,"Сетевой инженер: кто такой, чем занимается и с...",https://netology.ru/blog/09-2023-who-is-networ...,Современные профессии
0,13 сентября 2023,Как работает команда контента в Нетологии,https://netology.ru/blog/09-2023-content-at-ne...,Нетология
0,30 августа 2023,Зачем нужна магистратура: как за два года учёб...,https://netology.ru/blog/08-2023-why-masters-d...,Разобраться


Напишем скрипт, который будет собирать новости с сайта Коммерсанта

In [ ]:
URL = 'https://www.kommersant.ru/search/results'
params = {
    'search_query': 'python'
}

In [ ]:
res = requests.get(URL, params)

In [ ]:
res.text

'\r\n<!DOCTYPE html>\r\n\r\n<html class="no-js"  lang="ru">\r\n<head>\r\n    <title>Коммерсантъ: последние новости России и мира</title>\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge, chrome=1" />\r\n    <meta charset="utf-8" />\r\n\r\n    <meta name="format-detection" content="telephone=no" />\r\n    <meta name="title" content="Коммерсантъ: последние новости России и мира" />\r\n    <meta name="description" content="Актуальные новости, объективный анализ и эксклюзивные комментарии о важнейших событиях и трендах" />\r\n    <meta name="keywords" content="Новости,Политика,Экономика,Бизнес,Финансы,Дело,Биржа,Рынок,Акции,Прогнозы,Критика,Интервью,Рейтинги,Документы,Деньги,Власть,Автопилот,Тематические страницы,Первые лица,Деловые новости,Мировая практика,Культура,Спорт,Weekend,Астрологический прогноз,Погода мира,Курсы валют ЦБ РФ" />\r\n\r\n    <meta name="yandex-verification" content="50df68945a519dbd" />\r\n\r\n    \r\n<meta name="viewport" content="width=device-width, initi

In [ ]:
soup = BeautifulSoup(res.text)
soup

<!DOCTYPE html>
<html class="no-js" lang="ru">
<head>
<title>Коммерсантъ: последние новости России и мира</title>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="Коммерсантъ: последние новости России и мира" name="title"/>
<meta content="Актуальные новости, объективный анализ и эксклюзивные комментарии о важнейших событиях и трендах" name="description"/>
<meta content="Новости,Политика,Экономика,Бизнес,Финансы,Дело,Биржа,Рынок,Акции,Прогнозы,Критика,Интервью,Рейтинги,Документы,Деньги,Власть,Автопилот,Тематические страницы,Первые лица,Деловые новости,Мировая практика,Культура,Спорт,Weekend,Астрологический прогноз,Погода мира,Курсы валют ЦБ РФ" name="keywords"/>
<meta content="50df68945a519dbd" name="yandex-verification"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="324580610921010" property="fb:app_id"/>
<meta content="website" propert

In [ ]:
# собираем все теги со ссылками на полные тексты новостей
refs = soup.find_all('a', class_='uho__link')
print(len(refs))
print(refs)

24
[<a class="uho__link uho__link--overlay" href="/doc/6312197?query=python" target="_blank"><span class="vam">«Цифровая Россия» запускает проект переподготовки участников СВО по ИТ-специальностям</span></a>, <a class="uho__link" href="/doc/6312197?query=python" target="_blank"> ...  нейронных сетей на <mark>Python</mark> с нуля. – Разработчик ...  для анализа данных. – <mark>Python</mark> для анализа данных ... </a>, <a class="uho__link uho__link--overlay" href="/doc/6311661?query=python" target="_blank"><span class="vam">Рынок ИТ-найма сыграл по новым правилам</span></a>, <a class="uho__link" href="/doc/6311661?query=python" target="_blank">Рекрутеры изменили подходы в подборе ИТ-специалистов</a>, <a class="uho__link" href="/doc/6311661?query=python" target="_blank"> ...  разработчики Java и <mark>Python</mark>. Особенно набирает обороты ... </a>, <a class="uho__link uho__link--overlay" href="/doc/6312170?query=python" target="_blank"><span class="vam">Нижегородских ветеранов СВО буд

In [ ]:
# добираемся до ссылок
all_links = []

for ref in refs:
    all_links.append(ref.get('href'))

print(len(all_links))
print(all_links)

24
['/doc/6312197?query=python', '/doc/6312197?query=python', '/doc/6311661?query=python', '/doc/6311661?query=python', '/doc/6311661?query=python', '/doc/6312170?query=python', '/doc/6312170?query=python', '/doc/6310903?query=python', '/doc/6310903?query=python', '/doc/6310903?query=python', '/doc/6296993?query=python', '/doc/6296993?query=python', '/doc/6296079?query=python', '/doc/6296079?query=python', '/doc/6282292?query=python', '/doc/6282292?query=python', '/doc/6187050?query=python', '/doc/6187050?query=python', '/doc/6110090?query=python', '/doc/6110090?query=python', '/doc/6110090?query=python', '/doc/6070089?query=python', '/doc/6070089?query=python', '/doc/6070089?query=python']


In [ ]:
# исключаем дубли
all_links = set(all_links)
print(len(all_links))
print(all_links)

10
{'/doc/6296993?query=python', '/doc/6312197?query=python', '/doc/6187050?query=python', '/doc/6311661?query=python', '/doc/6070089?query=python', '/doc/6110090?query=python', '/doc/6296079?query=python', '/doc/6312170?query=python', '/doc/6310903?query=python', '/doc/6282292?query=python'}


In [ ]:
# формируем полноценные ссылки
all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))
print(all_full_links)

['https://www.kommersant.ru/doc/6296993?query=python', 'https://www.kommersant.ru/doc/6312197?query=python', 'https://www.kommersant.ru/doc/6187050?query=python', 'https://www.kommersant.ru/doc/6311661?query=python', 'https://www.kommersant.ru/doc/6070089?query=python', 'https://www.kommersant.ru/doc/6110090?query=python', 'https://www.kommersant.ru/doc/6296079?query=python', 'https://www.kommersant.ru/doc/6312170?query=python', 'https://www.kommersant.ru/doc/6310903?query=python', 'https://www.kommersant.ru/doc/6282292?query=python']


In [ ]:
# объединим все в одну функцию
def get_all_links(query):
    url = 'https://www.kommersant.ru/search/results'
    params = {
        'search_query': query
    }
    res = requests.get(url, params)
    soup = BeautifulSoup(res.text)
    refs = soup.find_all('a', class_='uho__link')

    all_links = []
    for ref in refs:
        all_links.append(ref.get('href'))

    all_links = set(all_links)

    all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))

    return all_full_links

all_links = get_all_links('python')
print(all_links)

['https://www.kommersant.ru/doc/6296993?query=python', 'https://www.kommersant.ru/doc/6312197?query=python', 'https://www.kommersant.ru/doc/6187050?query=python', 'https://www.kommersant.ru/doc/6311661?query=python', 'https://www.kommersant.ru/doc/6070089?query=python', 'https://www.kommersant.ru/doc/6110090?query=python', 'https://www.kommersant.ru/doc/6296079?query=python', 'https://www.kommersant.ru/doc/6312170?query=python', 'https://www.kommersant.ru/doc/6310903?query=python', 'https://www.kommersant.ru/doc/6282292?query=python']


In [ ]:
# но мы же собрали только одну страницу? Хотим ВСЕ новости
def get_all_links(query, pages):
    url = 'https://www.kommersant.ru/search/results'
    links_list = []
    params = {
        'search_query': query
    }
    for i in range(1, pages+1):
        params['page'] = i
        res = requests.get(url, params)
        soup = BeautifulSoup(res.text)
        # print(res.text)
        time.sleep(0.3)

        refs = soup.find_all('a', class_='uho__link')
        all_links = []

        for ref in refs:
            all_links.append(ref.get('href'))

        all_links = set(all_links)

        all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))

        links_list += all_full_links
    return links_list

all_links = get_all_links('python', 3)
print(len(all_links))
print(all_links)

27
['https://www.kommersant.ru/doc/6296993?query=python', 'https://www.kommersant.ru/doc/6312197?query=python', 'https://www.kommersant.ru/doc/6187050?query=python', 'https://www.kommersant.ru/doc/6311661?query=python', 'https://www.kommersant.ru/doc/6070089?query=python', 'https://www.kommersant.ru/doc/6110090?query=python', 'https://www.kommersant.ru/doc/6296079?query=python', 'https://www.kommersant.ru/doc/6312170?query=python', 'https://www.kommersant.ru/doc/6310903?query=python', 'https://www.kommersant.ru/doc/6282292?query=python', 'https://www.kommersant.ru/doc/5898889?query=python', 'https://www.kommersant.ru/doc/5952039?query=python', 'https://www.kommersant.ru/doc/5941884?query=python', 'https://www.kommersant.ru/gallery/2747165?query=python&sids=1159624', 'https://www.kommersant.ru/doc/5293079?query=python', 'https://www.kommersant.ru/doc/5842661?query=python', 'https://www.kommersant.ru/doc/5926912?query=python', 'https://www.kommersant.ru/doc/5900151?query=python', 'https:

In [ ]:
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
    print(date)
    title = soup.find('h1', class_='doc_header__name').text.strip()
    print(title)
    text = soup.find('div', class_='doc__body').text.strip()
    print(text)

In [ ]:
# обернем в функцию
def get_kom_news(links):
    kom_news = pd.DataFrame()
    for link in all_links:
        soup = BeautifulSoup(requests.get(link).text)
        time.sleep(0.3)
        date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
        title = soup.find('h1', class_='doc_header__name').text.strip()
        text = soup.find('div', class_='doc__body').text.strip().replace('\n', '')
        row = {'date': date, 'title': title, 'text': text}
        kom_news = pd.concat([kom_news, pd.DataFrame([row])])
    return kom_news.reset_index(drop=True)

In [ ]:
kom_news = get_kom_news(all_links)
kom_news

,date,title,text
0,2023-10-24 16:30:10+03:00,Какой язык программирования выбрать в 2024 году,Какой язык программирования выбрать в 2024 год...
1,2023-11-01 13:41:41+03:00,«Цифровая Россия» запускает проект переподгото...,«Цифровая Россия» запускает проект переподгото...
2,2023-08-31 20:14:20+03:00,"Старшеклассникам расскажут, заменит ли их иску...","Старшеклассникам расскажут, заменит ли их иску..."
3,2023-11-01 13:13:53+03:00,Рынок ИТ-найма сыграл по новым правилам,Рынок ИТ-найма сыграл по новым правилам\r ...
4,2023-06-30 15:02:22+03:00,"«Даже ChatGPT по сути научный прорыв, еще не п...","«Даже ChatGPT по сути научный прорыв, еще не п..."
5,2023-07-17 17:24:26+03:00,«Без людей никакие технологии внедряться не бу...,«Без людей никакие технологии внедряться не бу...
6,2023-10-24 00:00:01+03:00,Не только «цифра»,Не только «цифра»\r Расширение инст...
7,2023-11-01 13:09:11+03:00,Нижегородских ветеранов СВО будут переучивать ...,Нижегородских ветеранов СВО будут переучивать ...
8,2023-10-30 21:25:58+03:00,Нейросетевая библиотека для быстрых аэродинами...,Нейросетевая библиотека для быстрых аэродинами...
9,2023-10-18 07:59:10+03:00,Жители Прикамья смогут обучиться аналитике данных,Жители Прикамья смогут обучиться аналитике дан...
